In [268]:
print('nyan!')

nyan!


In [1]:
import pandas as pd
import time
import re
import tweepy
import requests
from bs4 import BeautifulSoup
from types import MappingProxyType
import datetime

# 2022-23のプレミアリーグ20クラブ
EPL_CODE = [
    'ブレントフォード','リヴァプール','エヴァートン','ブライトン','レスター','フラム','アーセナル','ニューカッスル',
    'マンチェスター・U','ボーンマス','サウサンプトン','フォレスト','リーズ','ウェストハム','アストン・ヴィラ',
    'ウルヴァーハンプトン','クリスタル・パレス','トッテナム','チェルシー','マンチェスター・C'
]

# BIG6 + ちょっと気になる
BIG6_CODE = [
    'アーセナル',
    'マンチェスター・U',
    'マンチェスター・C',
    'トッテナム',
    'チェルシー',
    'リヴァプール',
    'ニューカッスル',
    'リーズ',
    'ブレントフォード'
]

# 曜日（英語 ⇒ 日本語）
WEEK_DAYS_DICT: dict = MappingProxyType({
        'Mon':'（月）',
        'Tue':'（火）',
        'Wed':'（水）',
        'Thu':'（木）',
        'Fri':'（金）',
        'Sat':'（土）',
        'Sun':'（日）'
})

In [26]:
def scrapeSportsNaviPage(t_yyyymmdd: str, t_en_week_name: str, y_yyyymmdd: str, y_en_week_name: str):
    """
    SportsNaviより引数で指定された開催日・曜日（英語）より、対戦相手および試合日時を取得する関数。
    """
    print('scrapeSportsNaviPage start')
    
    # 曜日の英語 ⇒ 日本語へ変換
    t_ja_week_name = WEEK_DAYS_DICT.get(t_en_week_name)
    y_ja_week_name = WEEK_DAYS_DICT.get(y_en_week_name)
    
    # 検索キーの作成
    t_search_key = str(int(t_yyyymmdd[4:6])) + '/' + str(int(t_yyyymmdd[6:8])) + t_ja_week_name
    y_search_key = str(int(y_yyyymmdd[4:6])) + '/' + str(int(y_yyyymmdd[6:8])) + y_ja_week_name

    # 対象ページ
    url = 'https://soccer.yahoo.co.jp/ws/category/eng'

    # スクレイピング
    top_r = requests.get(url)

    # HTMLを解析する
    top_soup = BeautifulSoup(top_r.content, "html.parser")

    # テーブル情報を取得
    td_list = top_soup.find\
            ('table', class_='sc-tableGame').find_all('td')

    # 日付
    match_datetime_list = []
    match_datetime_list_ans = [] #空文字削除後
    
    # 節
    match_kinds_list = []

    # チーム
    team_name_list = []
    
    for td in td_list:
            # 検索対象の試合日時の場合（00:00～24:00）
            # print(td.text)
            if t_search_key in td.text:
                work_td = td.text.split('\n')
                for name in work_td:
                    if re.search(r'(\d*):(\d*)', name) is not None:
                        match_datetime_list.append(name)
                        
            # 検索対象の試合日時の場合（24:01～）
            if y_search_key in td.text:
                work_td = td.text.split('\n')
                for name in work_td:
                    if re.search(r'(\d*):(\d*)', name) is not None:
                        match_datetime_list.append(name)

                        
            # クラブ名を取得
            for num in range(len(EPL_CODE)):
                if EPL_CODE[num] in td.text:
                    team_name_list.append(EPL_CODE[num])
            
    # クラブ名リストの空文字を削除
    team_name_list = [a for a in team_name_list if a != '']    

    print('scrapeSportsNaviPage end')
    
    for name in match_datetime_list:
        #print(name)
        name = name.lstrip()
        #print(name)
        match_datetime_list_ans.append(name)
        
    match_day_list = []
    match_day_list.append(t_search_key)
    match_day_list.append(y_search_key)
        
    return match_day_list, match_datetime_list_ans, team_name_list, ;

In [30]:
# 現在時刻の取得
today = datetime.datetime.now() 
t_yyyymmdd = today.strftime("%Y%m%d")
t_yyyymmdd_week = today.strftime("%a")

print(today, t_yyyymmdd, t_yyyymmdd_week)

# 操作後の日付取得
yesterday = today + datetime.timedelta(-1)
y_yyyymmdd = yesterday.strftime("%Y%m%d")
y_yyyymmdd_week = yesterday.strftime("%a")

print(yesterday, y_yyyymmdd, y_yyyymmdd_week)

2023-01-04 22:43:07.192392 20230104 Wed
2023-01-03 22:43:07.192392 20230103 Tue


In [13]:
# 関数呼び出し
match_info_list = []
match_info_list = scrapeSportsNaviPage(t_yyyymmdd[0:8], t_yyyymmdd_week, y_yyyymmdd[0:8], y_yyyymmdd_week)
print(match_info_list)

scrapeSportsNaviPage start
scrapeSportsNaviPage end
(['1/4（水）', '1/3（火）'], ['28:30', '28:45', '29:00', '29:00'], ['サウサンプトン', 'フォレスト', 'リーズ', 'ウェストハム', 'アストン・ヴィラ', 'ウルヴァーハンプトン', 'クリスタル・パレス', 'トッテナム'])


In [36]:
def createMatchInfoList(match_info_list):
    """
    SportsNaviより取得したmatch_info_listを加工してTwitterで配信する際の文字列を作成する関数。
    """
    print('createMatchInfoList start')
    
    # ホーム＆アウェーで仕分け
    home_team_name_list = []
    away_team_name_list = []

    for num in range(len(match_info_list[2])):
        if num % 2 == 0:
            home_team_name_list.append(match_info_list[2][num])
        elif num % 2 != 0:
            away_team_name_list.append(match_info_list[2][num])
    
    
    # BIG6リストに存在すれば保持する
    home_team_name_list_ans = []
    away_team_name_list_ans = []

    for num in range(len(away_team_name_list)):
        for big6num in range(len(BIG6_CODE)):
            if away_team_name_list[num] == BIG6_CODE[big6num]:
                away_team_name_list_ans.append(num)

    for num in range(len(home_team_name_list)):
        for big6num in range(len(BIG6_CODE)):
            if home_team_name_list[num] == BIG6_CODE[big6num]:
                home_team_name_list_ans.append(num)
                
    
    # 重複を排除×昇順ソート
    club_index = []

    for num in range(len(home_team_name_list_ans)):
        club_index.append(home_team_name_list_ans[num])
    for num in range(len(away_team_name_list_ans)):
        club_index.append(away_team_name_list_ans[num])
    club_index = set(club_index)
    club_index = sorted(club_index)
    

    # ホームチーム
    home_club = []

    for num in range(len(club_index)):
        home_club.append(home_team_name_list[club_index[num]])


    # アウェイチーム
    away_club = []

    for num in range(len(club_index)):
        away_club.append(away_team_name_list[club_index[num]])
        

    # 試合時間
    match_datetime = []

    for num in range(len(club_index)):
        match_datetime.append(match_info_list[1][club_index[num]])

    # TODO：試合日程
    # 今後は日付を正確にする
    # https://soccer.yahoo.co.jp/ws/category/eng の情報配信時刻が不明なのよ
    # match_day = []

    # print(match_datetime[num][0:2])

    # for num in range(len(match_datetime)):
    #     if match_datetime[num][0:2] >= 24:
    #         match_day.append()
    
    print('createMatchInfoList end')
    
    return match_datetime, home_club, away_club

In [37]:
match_list = createMatchInfoList(match_info_list)
print(match_list)

createMatchInfoList start
createMatchInfoList end
(['28:45', '29:00'], ['リーズ', 'クリスタル・パレス'], ['ウェストハム', 'トッテナム'])


In [235]:
# 呼び出してから加工するのはここからにする
# BIG6_CODE test
print(BIG6_CODE)

search_key = 'アーセナル'

if search_key in BIG6_CODE:
    print('もい！')

['アーセナル', 'マンチェスター・U', 'マンチェスター・ユナイテッド', 'マンチェスター・C', 'マンチェスター・シティー', 'トッテナム', 'トッテナム・ホットスパー', 'チェルシー', 'リヴァプール', 'ニューカッスル', 'ニューカッスル・ユナイテッド', 'リーズ', 'リーズ・ユナイテッド', 'ブレントフォード']
もい！


In [286]:
# 没
ans_len = max(len(home_team_name_list_ans), len(away_team_name_list_ans))
print(ans_len)


            

2


TypeError: list.append() takes exactly one argument (0 given)

In [287]:
# 降順ソート
new_list_reverse = sorted(ans, reverse=True)

print(new_list_reverse)

[3, 2]


In [288]:
# 昇順ソート
new_list_reverse = sorted(ans)

print(new_list_reverse)

[2, 3]
